In [129]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

In [130]:
raw_merged = pd.read_csv('zillow_raw.csv')

### Dealing with datetime variables

In [131]:
raw_merged['transactiondate']=pd.to_datetime(raw_merged['transactiondate'])
raw_merged['month'] = raw_merged['transactiondate'].dt.month
raw_merged['year'] = raw_merged['transactiondate'].dt.year

raw_merged.drop('transactiondate', axis=1, inplace=True)


### Dealing with missing variables - Part 1

#### Pools and Hot Tubs

In [132]:
# 'NaN' in poolcnt means '0 pools'
raw_merged['poolcnt'].fillna(0, inplace=True)

# 'NaN' in 'hashottuborspa' means 0 and convert True to 1
raw_merged['hashottuborspa'].fillna(0, inplace=True)
raw_merged ['hashottuborspa'].replace(True, 1, inplace=True)

#Fill missing poolsize column for properties that have a pool with the median value of the poolsize
raw_merged.loc[raw_merged['poolcnt']>0, 'poolsizesum'] =  raw_merged.loc[raw_merged['poolcnt']>0, 'poolsizesum'].fillna(
                                                            raw_merged[raw_merged['poolcnt']>0]['poolsizesum'].median())

# if no pool, then poosizesum is 0
raw_merged.loc[raw_merged['poolcnt']==0, 'poolsizesum']=0

# Drop redundant variables regarding pools and hot tubs
raw_merged.drop(['pooltypeid2', 'pooltypeid7', 'pooltypeid10'], axis=1, inplace=True)

#### Fireplace 

In [133]:
# If 'fireplaceflag' is 'NaN' we will replace it with 0, and if its 'True', we will replace it with 1
raw_merged['fireplaceflag'].fillna(0, inplace=True)
raw_merged['fireplaceflag'].replace(True, 1, inplace=True)

#Drop 'fireplacecnt' because fireplaceflag provides the same information
raw_merged.drop('fireplacecnt', axis=1, inplace=True)



#### Garage

In [134]:
# if 'garagecarcnt' is missing, fill with 0 and do same with 'garagetotalsqft'
raw_merged['garagecarcnt'].fillna(0, inplace=True)
raw_merged['garagetotalsqft'].fillna(0, inplace=True)

#### Tax Deliquency

In [135]:
# if 'taxdeliquencyflag' is missing , fill with 0 and if its 'True' we will replace it with 1
raw_merged['taxdelinquencyflag'].fillna(0, inplace=True)
raw_merged['taxdelinquencyflag'].replace('Y', 1, inplace=True)

#Drop the taxdeliquencyyear as it gives the same information as above
raw_merged.drop('taxdelinquencyyear', axis=1, inplace=True)

#### Basement & Deck

In [136]:
# replace 'nan' of basementsqft to 0
raw_merged['basementsqft'].fillna(0, inplace=True)

# if 'decktypeid' is missing, replace it to 0 and change all other type to 1
raw_merged['decktypeid'].fillna(0, inplace=True)
raw_merged['decktypeid'] = np.where(raw_merged['decktypeid']==0, 0, 1)


#### Aircondition type and Heating type

In [137]:
#AIRCONDITION

# if 'airconditiontypeid' is 'NaN' change to 5
raw_merged['airconditioningtypeid'].fillna(5, inplace=True)

# Change airconditiontypeid to 0 if it has no AC, 1 if it has central (already in data) and 2 if anything else
#raw_merged['airconditioningtypeid'] = raw_merged['airconditioningtypeid'].apply(lambda x:[0 if y==5 else y for y in x])
#raw_merged['airconditioningtypeid'] = raw_merged['airconditioningtypeid'].apply(lambda x:[y if y==0 | y ==1 else 2 for y in x ])

raw_merged['airconditioningtypeid'] = np.where(raw_merged['airconditioningtypeid']==5, 0 , raw_merged['airconditioningtypeid'])
raw_merged['airconditioningtypeid'] = np.where((raw_merged['airconditioningtypeid']==0) | (raw_merged['airconditioningtypeid']==1)
                                      ,raw_merged['airconditioningtypeid'], 2)

#HEATING

# if 'heatingtypeid' is 'NaN' change to 13
raw_merged['heatingorsystemtypeid'].fillna(13, inplace=True)

# Change 'heatingtypeid' to 0 is none, 1 to central and 2 to all others
raw_merged['heatingorsystemtypeid'] = np.where(raw_merged['heatingorsystemtypeid']==13, 0, raw_merged['heatingorsystemtypeid'])
raw_merged['heatingorsystemtypeid'] = np.where((raw_merged['heatingorsystemtypeid']==0) | (raw_merged['heatingorsystemtypeid']==1)
                                      ,raw_merged['heatingorsystemtypeid'], 2)
                                                                                                   


#### Patio and Yard shed

In [138]:
# If patio is 'NaN' replace with 0, all other sqft replace as 1 to indicate has patio
raw_merged['yardbuildingsqft17'].fillna(0, inplace=True)
raw_merged['hasPatio'] = np.where(raw_merged['yardbuildingsqft17']==0, 0, 1)

#If shed is 'NaN' replace with 0, all other sqft replace as 1 to indicate presence of shed
raw_merged['yardbuildingsqft26'].fillna(0, inplace=True)
raw_merged['hasShed'] = np.where(raw_merged['yardbuildingsqft26'] ==0, 0, 1)

#Drop the two variables
raw_merged.drop(['yardbuildingsqft17', 'yardbuildingsqft26'], axis=1, inplace=True)

### Dropping redundant variables

In [139]:
## Getting rid of similiar features to avoid multicolinearity (based on graphs for colinearity)

sqft_redundant = ['finishedsquarefeet12', 'finishedfloor1squarefeet', 'finishedsquarefeet6', 'finishedsquarefeet12', 
                 'finishedsquarefeet13', 'finishedsquarefeet15', 'finishedsquarefeet50']

# Variable 'bathroomcnt' encompasses all the bathroom variables
bathroom_redundant = ['threequarterbathnbr', 'fullbathcnt', 'calculatedbathnbr']

# location variables are already given by latitude and longitude
location_redundant = ['rawcensustractandblock', 'censustractandblock', 'regionidneighborhood', 'regionidcity', 'regionidzip']

# tax redundant values
tax_redundant = ['landtaxvaluedollarcnt', 'structuretaxvaluedollarcnt', 'assessmentyear']

#other redundant variables
other_redundant = ['storytypeid', 'architecturalstyletypeid', 'buildingclasstypeid', 'typeconstructiontypeid' ]

#drop variables
variables_to_drop = sqft_redundant + bathroom_redundant + location_redundant + tax_redundant + other_redundant

raw_merged.drop( variables_to_drop, axis=1, inplace=True)

## Dealing with missing variable - Part 2

In [140]:
raw_merged.isna().sum().sort_values()

parcelid                            0
year                                0
month                               0
taxdelinquencyflag                  0
fireplaceflag                       0
roomcnt                             0
regionidcounty                      0
propertylandusetypeid               0
poolsizesum                         0
hasPatio                            0
longitude                           0
latitude                            0
heatingorsystemtypeid               0
poolcnt                             0
garagetotalsqft                     0
logerror                            0
airconditioningtypeid               0
basementsqft                        0
bathroomcnt                         0
hashottuborspa                      0
bedroomcnt                          0
hasShed                             0
decktypeid                          0
fips                                0
garagecarcnt                        0
propertycountylandusecode           1
taxvaluedoll

In [141]:
# replace 'numberofstorieswith the most common observation
raw_merged['numberofstories'].fillna((raw_merged['numberofstories'].mode()[0]), inplace=True)

#replace buildingqualitytypeid with the most common observation
raw_merged['buildingqualitytypeid'].fillna((raw_merged['buildingqualitytypeid'].mode()[0]), inplace=True)

#propertyzoning with the most common observation
raw_merged['propertyzoningdesc'].fillna((raw_merged['propertyzoningdesc'].mode()[0]), inplace=True)

#untcnt with the most common
raw_merged['unitcnt'].fillna((raw_merged['unitcnt'].mode()[0]), inplace=True)

#lotsize squarefeet with the average value
raw_merged['lotsizesquarefeet'].fillna((raw_merged['lotsizesquarefeet'].mean()), inplace=True)

#yearbuilt with the most common
raw_merged['yearbuilt'].fillna((raw_merged['yearbuilt'].mode()[0]), inplace=True)

#calculatedfinishedsqft with the median
raw_merged['calculatedfinishedsquarefeet'].fillna((raw_merged['calculatedfinishedsquarefeet'].median()), inplace=True)

#taxamount with median
raw_merged['taxamount'].fillna((raw_merged['taxamount'].median()), inplace=True)

#propertylanduse with common observation
raw_merged['propertycountylandusecode'].fillna((raw_merged['propertycountylandusecode'].mode()[0]), inplace=True)

#tax value dollar count with median
raw_merged['taxvaluedollarcnt'].fillna((raw_merged['taxvaluedollarcnt'].median()), inplace=True)

In [142]:
raw_merged.isna().sum().sort_values()

parcelid                        0
propertycountylandusecode       0
propertylandusetypeid           0
propertyzoningdesc              0
regionidcounty                  0
roomcnt                         0
unitcnt                         0
poolsizesum                     0
yearbuilt                       0
fireplaceflag                   0
taxvaluedollarcnt               0
taxamount                       0
taxdelinquencyflag              0
month                           0
year                            0
numberofstories                 0
hasPatio                        0
poolcnt                         0
longitude                       0
logerror                        0
airconditioningtypeid           0
basementsqft                    0
bathroomcnt                     0
bedroomcnt                      0
buildingqualitytypeid           0
lotsizesquarefeet               0
decktypeid                      0
fips                            0
garagecarcnt                    0
garagetotalsqf

In [143]:
raw_merged.shape

(90275, 35)

## Feature Engineering

In [144]:
# Tax Variables
raw_merged['taxpercentage'] = raw_merged['taxamount'] / raw_merged['taxvaluedollarcnt']
raw_merged.drop(['taxamount', 'taxvaluedollarcnt'], axis=1, inplace=True)

#Create dummy variables


In [145]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

raw_merged['propertycountylandusecode'] = le.fit_transform(raw_merged['propertycountylandusecode'])
raw_merged['propertyzoningdesc'] = le.fit_transform(raw_merged['propertyzoningdesc'])


In [146]:
raw_merged.to_csv('ModelReadyZillow.csv', index=False)